In [1]:
%config Completer.use_jedi = False

In [8]:
from census.getCensus import getCensus
from census.variables.models import GroupCode
from census.models import GeoDomain
import pandas as pd

In [3]:
c = getCensus(2019, shouldCacheOnDisk=True, shouldLoadFromExistingCache=True)

In [4]:
c.getGroups().head()

,code,description
0,B01001,SEX BY AGE
1,B01001A,SEX BY AGE (WHITE ALONE)
2,B01001B,SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)
3,B01001C,SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)
4,B01001D,SEX BY AGE (ASIAN ALONE)


In [5]:
variables = c.getVariablesByGroup(
    c.groups.EducationalAttainmentForThePopulation25YearsAndOver.code,
    c.groups.EducationalAttainmentByPresenceOfAComputerAndTypesOfInternetSubscriptionInHousehold.code,
    c.groups.EmploymentStatusForThePopulation16YearsAndOver.code,
    c.groups.HealthInsuranceCoverageStatusAndTypeByAgeByEducationalAttainment.code,
    c.groups.HealthInsuranceCoverageStatusAndTypeByEmploymentStatus.code,
)

  0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
educVars = c.getVariablesByGroup(c.groups.EducationalAttainmentForThePopulation25YearsAndOver.code)
educAndCompVars = c.getVariablesByGroup(c.groups.EducationalAttainmentByPresenceOfAComputerAndTypesOfInternetSubscriptionInHousehold.code)
emplVars = c.getVariablesByGroup(c.groups.EmploymentStatusForThePopulation16YearsAndOver.code)
healthInsByEduc = c.getVariablesByGroup(c.groups.HealthInsuranceCoverageStatusAndTypeByAgeByEducationalAttainment.code)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
educData = c.getStats(educVars['code'].tolist(), GeoDomain('congressional district'), [GeoDomain('state')], replaceColumnHeaders=True)
emplData = c.getStats(emplVars['code'].tolist(), GeoDomain('congressional district'), )

|          | 0/? [00:00<?, ?it/s]

In [10]:
educData.head(n=2)

,NAME,congressional district,state,Estimate_Total,AnnotationOfEstimate_Total,MarginOfError_Total,AnnotationOfMarginOfError_Total,Estimate_Total_NoSchoolingCompleted,AnnotationOfEstimate_Total_NoSchoolingCompleted,MarginOfError_Total_NoSchoolingCompleted,...,MarginOfError_Total_MastersDegree,AnnotationOfMarginOfError_Total_MastersDegree,Estimate_Total_ProfessionalSchoolDegree,AnnotationOfEstimate_Total_ProfessionalSchoolDegree,MarginOfError_Total_ProfessionalSchoolDegree,AnnotationOfMarginOfError_Total_ProfessionalSchoolDegree,Estimate_Total_DoctorateDegree,AnnotationOfEstimate_Total_DoctorateDegree,MarginOfError_Total_DoctorateDegree,AnnotationOfMarginOfError_Total_DoctorateDegree
0,"Congressional District 3 (116th Congress), California",03,06,492041,None,6040,None,9757,None,1948,...,3477,None,9976,None,1574,None,6172,None,1214,None
1,"Congressional District 21 (116th Congress), California",21,06,424849,None,10792,None,21911,None,3774,...,1546,None,3547,None,1356,None,889,None,422,None


In [18]:
educData.columns

Index(['NAME', 'congressional district', 'state', 'Estimate_Total',
       'Estimate_Total_NoSchoolingCompleted',
       'Estimate_Total_NurseryTo4thGrade', 'Estimate_Total_5thAnd6thGrade',
       'Estimate_Total_7thAnd8thGrade', 'Estimate_Total_9thGrade',
       'Estimate_Total_10thGrade', 'Estimate_Total_11thGrade',
       'Estimate_Total_12thGradeNoDiploma',
       'Estimate_Total_RegularHighSchoolDiploma',
       'Estimate_Total_GedOrAlternativeCredential',
       'Estimate_Total_SomeCollegeLessThan1Year',
       'Estimate_Total_SomeCollege1OrMoreYearsNoDegree',
       'Estimate_Total_AssociatesDegree', 'Estimate_Total_BachelorsDegree',
       'Estimate_Total_MastersDegree',
       'Estimate_Total_ProfessionalSchoolDegree',
       'Estimate_Total_DoctorateDegree'],
      dtype='object')

In [19]:
educPcts = educData.loc[:,['NAME', 'congressional district', 'state']]

In [20]:
educPcts['pctNoHs'] = (educData.Estimate_Total_NoSchoolingCompleted \
                    + educData.Estimate_Total_NurseryTo4thGrade \
                    + educData.Estimate_Total_5thAnd6thGrade \
                    + educData.Estimate_Total_7thAnd8thGrade \
                    + educData.Estimate_Total_9thGrade \
                    + educData.Estimate_Total_10thGrade \
                    + educData.Estimate_Total_11thGrade \
                    + educData.Estimate_Total_12thGradeNoDiploma) / educData.Estimate_Total
educPcts['pctHs'] = (educData.Estimate_Total_RegularHighSchoolDiploma \
                     + educData.Estimate_Total_GedOrAlternativeCredential) \
                        / educData.Estimate_Total
educPcts['pctBach'] = educData.Estimate_Total_BachelorsDegree / educData.Estimate_Total
educPcts['pctSomeCollege1YrNoDegree'] = educData.Estimate_Total_SomeCollegeLessThan1Year / educData.Estimate_Total
educPcts['pctSomeCollegeManyYrsNoDegree'] = educData.Estimate_Total_SomeCollege1OrMoreYearsNoDegree / educData.Estimate_Total
educPcts['pctGradPlus'] = (educData.Estimate_Total_DoctorateDegree + \
                          educData.Estimate_Total_MastersDegree ) / educData.Estimate_Total

In [21]:
c.searchGroups('employment')

,code,description
0,B10058,EMPLOYMENT STATUS OF GRANDPARENTS LIVING WITH OWN GRANDCHILDREN UNDER 18 YEARS BY RESPONSIBILITY FOR OWN GRANDCHILDREN AND AGE OF GRANDPARENT
1,B14005,SEX BY SCHOOL ENROLLMENT BY EDUCATIONAL ATTAINMENT BY EMPLOYMENT STATUS FOR THE POPULATION 16 TO 19 YEARS
2,B16010,EDUCATIONAL ATTAINMENT AND EMPLOYMENT STATUS BY LANGUAGE SPOKEN AT HOME FOR THE POPULATION 25 YEARS AND OVER
3,B17005,POVERTY STATUS IN THE PAST 12 MONTHS OF INDIVIDUALS BY SEX BY EMPLOYMENT STATUS
4,B18120,EMPLOYMENT STATUS BY DISABILITY STATUS AND TYPE
5,B19053,SELF-EMPLOYMENT INCOME IN THE PAST 12 MONTHS FOR HOUSEHOLDS
6,B19063,AGGREGATE SELF-EMPLOYMENT INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) FOR HOUSEHOLDS
7,B21005,AGE BY VETERAN STATUS BY EMPLOYMENT STATUS FOR THE CIVILIAN POPULATION 18 TO 64 YEARS
8,B23001,SEX BY AGE BY EMPLOYMENT STATUS FOR THE POPULATION 16 YEARS AND OVER
9,B23002A,SEX BY AGE BY EMPLOYMENT STATUS FOR THE POPULATION 16 YEARS AND OVER (WHITE ALONE)


In [22]:
employmentVars = c.getVariablesByGroup([c.groups.EmploymentStatusForThePopulation16YearsAndOver.code])

  0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
employmentByEducVars = c.getVariablesByGroup([c.groups.EducationalAttainmentByEmploymentStatusForThePopulation25To64Years.code])

  0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
healthInsuranceByEmploymentVars = c.getVariablesByGroup([c.groups.HealthInsuranceCoverageStatusAndTypeByEmploymentStatus.code])

  0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
filteredEmplVars = employmentVars[employmentVars['name'].str.contains('^Estimate')]

In [26]:
emplStats = c.getStats(filteredEmplVars['code'].tolist(), forDomain=GeoDomain('congressional district'), inDomains=[GeoDomain('state')], replaceColumnHeaders=True)

|          | 0/? [00:00<?, ?it/s]

In [27]:
emplStats.columns.to_list()

['NAME',
 'congressional district',
 'state',
 'Estimate_Total',
 'Estimate_Total_InLaborForce',
 'Estimate_Total_InLaborForce_CivilianLaborForce',
 'Estimate_Total_InLaborForce_CivilianLaborForce_Employed',
 'Estimate_Total_InLaborForce_CivilianLaborForce_Unemployed',
 'Estimate_Total_InLaborForce_ArmedForces',
 'Estimate_Total_NotInLaborForce']

In [28]:
emplVars = c.groupToVarsMapping.EmploymentStatusForThePopulation16YearsAndOver

In [31]:
emplVars.Estimate_Total.name

'Estimate_Total'

In [32]:
emplStats.columns

Index(['NAME', 'congressional district', 'state', 'Estimate_Total',
       'Estimate_Total_InLaborForce',
       'Estimate_Total_InLaborForce_CivilianLaborForce',
       'Estimate_Total_InLaborForce_CivilianLaborForce_Employed',
       'Estimate_Total_InLaborForce_CivilianLaborForce_Unemployed',
       'Estimate_Total_InLaborForce_ArmedForces',
       'Estimate_Total_NotInLaborForce'],
      dtype='object')

In [43]:
emplPcts = emplStats.loc[:,['NAME', 'congressional district', 'state']]

emplPcts['pctInLaborForce'] = (emplStats[emplVars.Estimate_Total_InLaborForce.name] /
                            emplStats[emplVars.Estimate_Total.name])
emplPcts['pctEmployed'] = (emplStats.Estimate_Total_InLaborForce_CivilianLaborForce_Employed /
                          emplStats.Estimate_Total_InLaborForce)
emplPcts['pctUnemployed'] = (emplStats.Estimate_Total_InLaborForce_CivilianLaborForce_Unemployed /
                            emplStats.Estimate_Total_InLaborForce_CivilianLaborForce)

In [45]:
emplPcts.head()

,NAME,congressional district,state,pctInLaborForce,pctEmployed,pctUnemployed
0,"Congressional District 3 (116th Congress), California",03,06,0.597684,0.934894,0.048259
1,"Congressional District 21 (116th Congress), California",21,06,0.565143,0.886146,0.093307
2,"Congressional District 48 (116th Congress), California",48,06,0.658193,0.955902,0.042958
3,"Congressional District 10 (116th Congress), California",10,06,0.610736,0.933713,0.066175
4,"Congressional District 34 (116th Congress), California",34,06,0.655622,0.942758,0.056410
